This is a demo notebook for the [bcbcpy](https://github.com/aheritianad/BootCamp-BlockChain-and-Python/) package.

In [1]:
try:
    import bcbcpy
except ModuleNotFoundError:
    import os, sys
    sys.path.append(os.path.abspath("../src/"))
    import bcbcpy

# Cryptography


In [2]:
from bcbcpy import crypto

## Hashing


### `Hash` function


$$\mathfrak{h}: X \to H$$

- $\mathfrak{h}(x) = \mathfrak{h}(y)$ *only if* $x=y$.
- given $h\in H$, it is *not easy* to find $x\in X$ such that $\mathfrak{h}(x) = h$.

In some sense, $\mathfrak{h}(x)$ is a signature of $x$.

In [3]:
text_to_be_hashed = "Hello World"
first_hash = crypto.hash_function(text_to_be_hashed, hash_name=crypto.HashName.sha256)
first_hash


'1370eeaaba7a6c7a234b1f82cc3b6d013a0088fd5e16408300f05b28b0015463'

In [4]:
crypto.hash_function(text_to_be_hashed + "!")


'e59f8bdf1305e382a4919ccefd613d3eebae612aa4c443f3af2d65663de3b075'

In [5]:
crypto.hash_function("Hello World!")

'e59f8bdf1305e382a4919ccefd613d3eebae612aa4c443f3af2d65663de3b075'

In [6]:
long_text = """
Hello everyone!

Welcome to this tutorial.
I promise that I will do my best to help you.

It will be interactive, so please participate as much as you can.

Please do not hesitate to ask if there you have questions.

Hope you will enjoy it!

Cheers!

Heritiana.
"""


In [7]:
hash_long = crypto.hash_function(long_text)
hash_long


'0c8359860bef1b26f811bf8448c546c2241c7c695f9b8ee76c1a8bf2d5b2dff3'

In [8]:
len(hash_long) == len(first_hash)


True

### Validation


In [9]:
difficulty = 4
crypto.is_valid_hash(first_hash, difficulty)


False

In [10]:
fake_hash = "0000" + first_hash[4:]
fake_hash

'0000eeaaba7a6c7a234b1f82cc3b6d013a0088fd5e16408300f05b28b0015463'

In [11]:
crypto.is_valid_hash(fake_hash, difficulty)


True

### Nonce


In [12]:
second_hash, nonce = crypto.hash_nonce_initializer(
    text_to_be_hashed, difficulty=difficulty
)
second_hash

'0000bfe6af4232f78b0c8eba37a6ba6c17b9b8671473b0b82305880be077edd9'

In [13]:
nonce


107105

In [14]:
crypto.is_valid_hash(second_hash, difficulty)


True

In [15]:
third_hash = crypto.hash_function(text_to_be_hashed, nonce=nonce)
third_hash


'0000bfe6af4232f78b0c8eba37a6ba6c17b9b8671473b0b82305880be077edd9'

In [16]:
second_hash == third_hash


True

In [17]:
forth_hash = crypto.hash_function(text_to_be_hashed, nonce, nonce="")  # type:ignore
second_hash == forth_hash

True

### Q: What if such `nonce` does not exist?


**Think about it.**


## Crypto


### `Encryption` & `Decryption`


$$f\circ f^{-1} (x) = f^{-1}\circ f(x) = x,\ \forall x$$


In [18]:
encrypt = lambda x: 2 * x - 1
decrypt = lambda x: 0  # TODO to fill

In [19]:
import random
try:
    for i in range(20):
        x = random.randint(-5000, 5000)
        assert (
            encrypt(decrypt(x)) == decrypt(encrypt(x)) == x
        ), f"Oops! fail at {i+1}-th attempt for x = {x}."
except AssertionError as e:
    print(e)
else:
    print("Congrats! I passes the test.")

Oops! fail at 1-th attempt for x = -2486.


In [20]:
clear_message = (
    "This is a clear message to encrypt and decrypt. I add this to make it longer."
)

### Types


#### Key


$\mathcal{F}=\{ f_\theta \:\ \theta \in \Theta\}$

$\left(f_{\theta}\right) ^{-1} = f_{\theta^\prime}$ for some $\theta^\prime \in \Theta$

$\theta \in \Theta$ : key


#### Symmetric


Given $f_\theta$ or $\theta\in \Theta$ (thus $f_\theta$), one can construct *easily*  $\left(f_\theta\right)^{-1}$.

From Wikipedia (2023-05-01):

*Symmetric-key algorithms are algorithms for cryptography that use the same cryptographic keys for both the encryption of plaintext and the decryption of ciphertext. The keys may be identical, or there may be a simple transformation to go between the two keys.*

##### Cesar


**Idea**: translation

$N$ : number of character.  
key: $ \theta \in  \mathbb{Z}$  
function: $f_\theta: x \mod N \mapsto (x + \theta) \mod N$  
inverse: $\theta^\prime = -\theta$

In [21]:
cesar_key = crypto.CesarKey.generate_key(max_length=2)
print(cesar_key)


CesarKey(2, -2)


In [22]:
crypto.CesarKey.compute_inverse(6)

-6

In [23]:
encr_cesar = cesar_key.encrypt(clear_message)

print(
    f"""
original:
========
{clear_message}

Encrypted:
========
{encr_cesar}
"""
)


original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Encrypted:
Vjku"ku"c"engct"oguucig"vq"gpet{rv"cpf"fget{rv0"K"cff"vjku"vq"ocmg"kv"nqpigt0



In [24]:
decr = cesar_key.decrypt(encr_cesar)

print(
    f"""
original:
========
{clear_message}

Decrypted:
=========
{decr}
"""
)


original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Decrypted:
This is a clear message to encrypt and decrypt. I add this to make it longer.



##### Permutation


In [25]:
crypto.PermutationKey.compute_inverse([2, 3, 4, 1])

[4, 1, 2, 3]

###### **Idea1: Chunking**

Chunk input with a chunk size equal to permutation length and apply permutation on each chunks.

In [26]:
perm_key = crypto.PermutationKey.generate_key(length=3, n_runs=1)
print(perm_key)


PermutationKey([3, 2, 1], [3, 2, 1])


In [27]:
encr_perm = perm_key.encrypt(clear_message)

print(
    f"""
original:
========
{clear_message}

Encrypted:
========
{encr_perm}
"""
)


original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Encrypted:
ihTi sa slc raeem ass eg otcnepyra t dncedpyr .ta I ddihtt sm oekati ol egnr.



In [28]:
decr = perm_key.decrypt(encr_perm)
print(
    f"""
original:
========
{clear_message}

Decrypted:
=========
{decr}
"""
)


original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Decrypted:
This is a clear message to encrypt and decrypt. I add this to make it longer.



###### **Idea2: Sliding**

One can think about sliding the permutation like the following.

Suppose we have the following permutation
$$\theta: \begin{cases}1\mapsto 3\\ 2\mapsto 2\\ 3\mapsto 1\end{cases}$$
and input `abcdefgh`.

- `input = abcdefgh`
- `abc defgh` $\to$ `cba defgh`
- `c bad efgh` $\to$ `c dab efgh`
- `cd abe fgh` $\to$ `cd eba fgh`
- `cde baf gh` $\to$ `cde fab gh`
- `cdef abg h` $\to$ `cdef gba h`
- `cdefg bah` $\to$ `cdefg hab`
- `output = cdefghab`

⚠️: the same process does not bring back `input` from output  
It can be fixed by doing a tricky thing to the *previous output* to get the *final output*.  
You can think about it and its implementation.

##### Adding noise


In [29]:
from bcbcpy import utils


In [30]:
noisy_message = utils.add_noises(clear_message)
noisy_message


"T|hpi[s- 7iGse +a< 4c}lxeVarrq jmUeysQs<aBg,eR {t$o+ Gekn/cmrWyCpNt% `a,n@dB /d$e8c<rZyBp^t\n., [IB 'a\\d.dM Ht)hmiHsK ~t^of Tmda4k1eg NiytU lleojnWg ewrq.3"

In [31]:
noisy_message[::2]

'This is a clear message to encrypt and decrypt. I add this to make it longer.'

In [32]:
utils.remove_noises(noisy_message)


'This is a clear message to encrypt and decrypt. I add this to make it longer.'

In [33]:
encr_noisy = cesar_key.encrypt(noisy_message)
decr_noisy = cesar_key.decrypt(encr_noisy)
decr_without_noise = utils.remove_noises(decr_noisy)

print(
    f"""
\t\t--- CESAR --
Original:
========
{clear_message}

Noisy message:
=============
{noisy_message}

Noisy Encrypted:
===============
{encr_noisy}

Noisy Decrypted:
===============
{decr_noisy}

Cleaned Decrypted:
=================
{decr_without_noise}
"""
)


		--- CESAR --
Original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Noisy message:
T|hpi[s- 7iGse +a< 4c}lxeVarrq jmUeysQs<aBg,eR {t$o+ Gekn/cmrWyCpNt% `a,n@dB /d$e8c<rZyBp^t
., [IB 'a\d.dM Ht)hmiHsK ~t^of Tmda4k1eg NiytU lleojnWg ewrq.3

Noisy Encrypted:
V~jrk]u/"9kIug"-c>"6e	nzgXctts"loWg{uSu>cDi.gT"}v&q-"Igmp1eotY{ErPv'"bc.pBfD"1f&g:e>t\{Dr`v!0."]KD")c^f0fO"Jv+jokJuM"
v`qh"Vofc6m3gi"Pk{vW"nngqlpYi"gyts05

Noisy Decrypted:
T|hpi[s- 7iGse +a< 4c}lxeVarrq jmUeysQs<aBg,eR {t$o+ Gekn/cmrWyCpNt% `a,n@dB /d$e8c<rZyBp^t
., [IB 'a\d.dM Ht)hmiHsK ~t^of Tmda4k1eg NiytU lleojnWg ewrq.3

Cleaned Decrypted:
This is a clear message to encrypt and decrypt. I add this to make it longer.



In [34]:
encr_noisy = perm_key.encrypt(noisy_message)
decr_noisy = perm_key.decrypt(encr_noisy)
decr_without_noise = utils.remove_noises(decr_noisy)

print(
    f"""
\t\t--- PERMUTATION ---
Original:
========
{clear_message}

Noisy message:
=============
{noisy_message}

Noisy Encrypted:
===============
{encr_noisy}

Noisy Decrypted:
===============
{decr_noisy}

Cleaned Decrypted:
=================
{decr_without_noise}
"""
)


		--- PERMUTATION ---
Original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Noisy message:
T|hpi[s- 7iGse +a< 4c}lxeVarrq jmUeysQs<aBg,eR {t$o+ Gekn/cmrWyCpNt% `a,n@dB /d$e8c<rZyBp^t
., [IB 'a\d.dM Ht)hmiHsK ~t^of Tmda4k1eg NiytU lleojnWg ewrq.3

Noisy Encrypted:
h|T[ip -sGi7 es<a+c4 xl}aVeqrrmj yeUsQsBa<e,g{ Ro$tG +nkemc/yWrNpC %t,a`d@n/ Be$d<c8yZr^pB.
t[ , BI\a'd.dH Mh)tHim Ks^t~ fodmTk4age1iN Utyll joegWnwe .qr3

Noisy Decrypted:
T|hpi[s- 7iGse +a< 4c}lxeVarrq jmUeysQs<aBg,eR {t$o+ Gekn/cmrWyCpNt% `a,n@dB /d$e8c<rZyBp^t
., [IB 'a\d.dM Ht)hmiHsK ~t^of Tmda4k1eg NiytU lleojnWg ewrq.3

Cleaned Decrypted:
This is a clear message to encrypt and decrypt. I add this to make it longer.



#### Asymmetric: Public-key cryptography


Knowing $f_\theta$ or $\theta\in \Theta$ (thus $f_\theta$) does **NOT** make the construction of $\left(f_\theta\right)^{-1}$ (thus $\theta^\prime$) *easy*.

From Wikipedia (2023-05-01):

*Public-key cryptography, or asymmetric cryptography, is the field of cryptographic systems that use pairs of related keys. Each key pair consists of a public key and a corresponding private key. Key pairs are generated with cryptographic algorithms based on mathematical problems termed one-way functions. Security of public-key cryptography depends on keeping the private key secret; the public key can be openly distributed without compromising security.*

##### Rivest–Shamir–Adleman aka RSA

([wikipedia](https://en.wikipedia.org/wiki/RSA_(cryptosystem)))

$\Theta \approx \mathbb{N} \times \mathbb{N}$  
public key: $\theta = (n,d)$  
private key: $\theta^\prime = (n,e)$  
function: $f_{(n,a)}: x\mod n \mapsto x^a \mod n$

**Construction**:  

1. Generate two primes $p,q$.
2. Compute $n=p\times q$.
3. Compute $\varphi(n) = (p-1)*(q-1)$.
4. Find $d$ such that $\gcd\left( d, \varphi(n)\right) = 1$.
5. Find $e$ such that $de + u\varphi(n)=1$ for some $u\in \mathbb{Z}$.
6. Discard $p,q,\varphi(n)$ and $u$ *(for security)*.

Note: [4.] and [5.] can be done easily and at once with the extended Euclidean algorithm.

**:warning: Security**: number factorization complexity

For large composite number $n$, computing $\varphi(n)$ is not *easy* without knowing its prime factors.

In [35]:
rsa_key = crypto.RSAPairKeys.generate_pairs(chunk_size=5)
print(rsa_key)

RSAPairKeys((7313, 4997), HIDDEN_KEY)


In [36]:
rsa_pub, rsa_priv = rsa_key
rsa_pub, rsa_priv

((7313, 4997), (7313, 713))

In [37]:
encr_rsa = rsa_key.encrypt(clear_message)
encr_rsa

'[\n"<rsachunkrsa>/8Q&j></rsachunkrsa>",\n"<rsachunkrsa>-z0NOf</rsachunkrsa>",\n"<rsachunkrsa>- OcF\\"</rsachunkrsa>",\n"<rsachunkrsa>2\\"gyi[</rsachunkrsa>",\n"<rsachunkrsa>D[_\\niH</rsachunkrsa>",\n"<rsachunkrsa>*t]H!\'</rsachunkrsa>",\n"<rsachunkrsa>,@d_hq</rsachunkrsa>",\n"<rsachunkrsa>D[I[nX</rsachunkrsa>",\n"<rsachunkrsa>-f)$k;</rsachunkrsa>",\n"<rsachunkrsa>8QgG-O</rsachunkrsa>",\n"<rsachunkrsa>D[Kbx7</rsachunkrsa>",\n"<rsachunkrsa>Foo\'r</rsachunkrsa>",\n"<rsachunkrsa>*tQXs </rsachunkrsa>",\n"<rsachunkrsa>$9@Ab(</rsachunkrsa>",\n"<rsachunkrsa>G<qgCv</rsachunkrsa>",\n"<rsachunkrsa>\\nKd</rsachunkrsa>"\n]'

In [38]:
decr_rsa = rsa_key.decrypt(encr_rsa)
decr_rsa


'[\n"<rsachunkrsa>This </rsachunkrsa>",\n"<rsachunkrsa>is a </rsachunkrsa>",\n"<rsachunkrsa>clear</rsachunkrsa>",\n"<rsachunkrsa> mess</rsachunkrsa>",\n"<rsachunkrsa>age t</rsachunkrsa>",\n"<rsachunkrsa>o enc</rsachunkrsa>",\n"<rsachunkrsa>rypt </rsachunkrsa>",\n"<rsachunkrsa>and d</rsachunkrsa>",\n"<rsachunkrsa>ecryp</rsachunkrsa>",\n"<rsachunkrsa>t. I </rsachunkrsa>",\n"<rsachunkrsa>add t</rsachunkrsa>",\n"<rsachunkrsa>his t</rsachunkrsa>",\n"<rsachunkrsa>o mak</rsachunkrsa>",\n"<rsachunkrsa>e it </rsachunkrsa>",\n"<rsachunkrsa>longe</rsachunkrsa>",\n"<rsachunkrsa>r.</rsachunkrsa>"\n]'

In [39]:
from bcbcpy.crypto import read_from_rsa

In [40]:
read_from_rsa(encr_rsa)

'/8Q&j>-z0NOf- OcF"2"gyi[D[_\niH*t]H!\',@d_hqD[I[nX-f)$k;8QgG-OD[Kbx7Foo\'r*tQXs $9@Ab(G<qgCv\nKd'

In [41]:
read_from_rsa(decr_rsa)

'This is a clear message to encrypt and decrypt. I add this to make it longer.'

In [42]:
rsa_key = crypto.RSAPairKeys.generate_pairs()
encr_rsa = rsa_key.encrypt(clear_message)
encr_rsa

'[\n"<rsachunkrsa>=#_GmL)M6SsjvN*{G(oK(czX9f`Nr$7:m~) !W1=evK\\n\\tz2!leC\\\\fHWw7-sTkvQ9ATd^/:i|RQ{WS*</rsachunkrsa>"\n]'

In [43]:
decr_rsa = rsa_key.decrypt(encr_rsa)
decr_rsa

'[\n"<rsachunkrsa>This is a clear message to encrypt and decrypt. I add this to make it longer.</rsachunkrsa>"\n]'

##### Diffie–Hellman key exchange

1. The network will:
   - choose a shared group $(G,*)$,
   - an element $g\in G$,
   - and compute $ord(g) \in \mathbb{N}$ for which is the smallest where $ord(g)\cdot g := \underbrace{g*g*\cdots*g}_{ord(g) \text{ times}}$ is equal to the neutral element of $G$.
2. For each $i$ in the network:
   - Choose $\theta^\prime_i \in \mathbb{N}$ such that $\theta^\prime_i < ord(g)$.  
   - Compute $f_{\theta_i} = \theta^\prime_i \dot g$.
   - Publish $f_{\theta_i}$ and keep $\theta^\prime_i$ secret.

If $i$ wants to send a message $m$ to $j$, he will proceed as follows:

1. retrieve $f_{\theta_j}$,
2. compute $P_{ij} := \theta_i \cdot f_{\theta_j}$,
3. encrypt $m$ with $P_{ij}$ (like the symmetric fashion),
4. send encrypted message $m_{ij}$ to $j$.

When $j$ receives $m_{ij}$, he will:

1. retrieve $f_{\theta_i}$,
2. compute $P_{ji} := \theta_j \cdot f_{\theta_i}$,
3. decrypt $m_{ij}$ with $P_{ji}$.
   
Note:
$$ \begin{align*}
P_{ij} 
&= \theta_i \cdot f_{\theta_j}&\\
&= \underbrace{f_{\theta_j}*\cdots*f_{\theta_j}}_{\theta_i \text{ times}}&\\
&= \underbrace{\left(\underbrace{g*\cdots*g}_{\theta_j \text{ times}}\right)*\cdots*\left(\underbrace{g*\cdots*g}_{\theta_j \text{ times}}\right)}_{\theta_i \text{ times}}&\\
&= \underbrace{g*\cdots*g}_{\theta_i\theta_j \text{ times}}&\\
&= \underbrace{\left(\underbrace{g*\cdots*g}_{\theta_j \text{ times}}\right)*\cdots*\left(\underbrace{g*\cdots*g}_{\theta_i \text{ times}}\right)}_{\theta_j \text{ times}}&\\
&= \underbrace{f_{\theta_j}*\cdots*f_{\theta_i}}_{\theta_j \text{ times}}&\\
&= \theta_j \cdot f_{\theta_i}&\\
&= P_{ji}&
\end{align*}$$

<p style="text-align: center;">
<img src="https://upload.wikimedia.org/wikipedia/commons/4/46/Diffie-Hellman_Key_Exchange.svg">
</p>

**⚠️ Security**: *discrete logarithm*

For some group ([Diffie-Hellman Group](https://www.ibm.com/docs/en/zos/2.3.0?topic=SSLTBW_2.3.0%2Fcom.ibm.tcp.ipsec.ipsec.help.doc%2Fcom%2Fibm%2Ftcp%2Fipsec%2Fipsec%2FPS_DT_Adv_PFS.CB_Initiate.html)), knowing the value of $g$ and $n \cdot g$ does not lead to $n$ *easily*.

In [44]:
# coming soon

##### Elliptic Curve Cryptography


[ECC](https://en.wikipedia.org/wiki/Elliptic-curve_cryptography) use the DH key exchange with Where the group $G$ is an [elliptic curve over finite fields](https://en.wikipedia.org/wiki/Elliptic_curve#Elliptic_curves_over_finite_fields).

In [45]:
# coming soon

### Communication & Security


#### Nodes generation

In [46]:
from bcbcpy.node import Node


In [47]:
key = crypto.RSAPairKeys.generate_pairs()
node = Node(key)
node.id

'user_87224'

In [48]:
node

In [49]:
class RepNode(Node):
    def __repr__(self) -> str:
        return f"{self.id} : {self._Node__keys}"  # type:ignore

In [50]:
akey = crypto.RSAPairKeys.generate_pairs()
alice = RepNode(akey, "Alice")
alice


Alice : RSAPairKeys((7313, 4349), HIDDEN_KEY)

In [51]:
bkey =  crypto.RSAPairKeys.generate_pairs()
bob = RepNode(username="Bob", keys=bkey)
bob

Bob : RSAPairKeys((7313, 2557), HIDDEN_KEY)

In [52]:
rkey = crypto.RSAPairKeys.generate_pairs()
random_guy = RepNode(rkey)
random_guy

user_518690 : RSAPairKeys((7313, 4877), HIDDEN_KEY)

#### Communication

In [53]:
sender = alice
receiver = bob

plain_message = (
    "Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)"
)

In [54]:
cipher = sender.encrypt(plain_message, key=receiver.pub)
out = cipher
if isinstance(receiver.pub, crypto.RSAKey):
    out = read_from_rsa(out)
out

"Fy|R5/pTP&v{ktMTfht8 Q&GG_|2^wU0o\\!*{U8DZYb,7,/8C 2Z$k1';3b=Wg-:kBC5EN,wE~#W"

In [55]:
out = receiver.decrypt(cipher)
if isinstance(sender.pub, crypto.RSAKey):
    out = crypto.read_from_rsa(out)
out

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

#### Attack

##### Reading attack


In [56]:
hacker = random_guy

In [57]:
out = hacker.decrypt(cipher)
if isinstance(hacker.pub, crypto.RSAKey):
    out = crypto.read_from_rsa(out)
out

'81/nJ(+44o1O,!>8ge4$3($J&P}L%)x|R.]Yd{,V;OS-?LJ~>\\<wa6c!uZ-iNzqeN`DN4YNr,]>t'

In [58]:
out = hacker.encrypt(cipher, receiver.pub)
if isinstance(receiver.pub, crypto.RSAKey):
    out = crypto.read_from_rsa(out)
out

'7Pq.pCdQ:57;aTz =x o%,m%Vmo\n54Z3 :.VK)FgNjr!n0CK3i/0/0$tZ3XcE_vzMA9A3TX:&\\OC'

##### Q: what if `receiver.pub == receiver.priv`?

In [59]:
_, receiver_priv = receiver._Node__keys
receiver_pub = receiver_priv # fake pub for testing

In [60]:
out = hacker.encrypt(cipher, receiver_pub)
if isinstance(receiver.pub, crypto.RSAKey):
    out = crypto.read_from_rsa(out)
# out

##### Sending attack


In [61]:
scam_message = (
    "Hello Bob! This is Alice. Please send me $1000 to this account: (0987654321)"
)

In [62]:
scam_cipher= hacker.encrypt(scam_message, key=receiver.pub)
out = scam_cipher
if isinstance(receiver.pub, crypto.RSAKey):
    out = crypto.read_from_rsa(out)
out

"Fy|R5/pTP&v{ktMTfht8 Q&GG_|2^wU0o\\!*{U8DZYb,7,/8C 2Z$k1';3b=Wg-:mQx[]G@kD1B-"

In [63]:
out = receiver.decrypt(scam_cipher)

if isinstance(sender.pub, crypto.RSAKey):
    out = crypto.read_from_rsa(out)
out

'Hello Bob! This is Alice. Please send me $1000 to this account: (0987654321)'

### Authentication


**Problem**: no way for the `receiver` to recognize the *true* `sender`.

#### Encryption

In [64]:
sender.sign(plain_message) == sender.decrypt(plain_message) # encrypt with sender.priv

True

In [65]:
signed_message = sender.sign(plain_message) 
cipher_signed = sender.encrypt(signed_message, key=receiver.pub)
cipher_signed

'[\n"<rsachunkrsa>\\"Psc\\t23&Fm(\\"OQU7NGZLu\\nIz7;i?\\\\F;<bmR\\n+%@l!0iR7\\"&qZ&?V`+f~8+v\\n]FAPKnZh\\t=7sbE8f</rsachunkrsa>"\n]'

In [66]:
with_noise = utils.add_noises(plain_message)
signed_with_noise = sender.sign(with_noise) 
cipher_signed_with_noise = sender.encrypt(signed_with_noise, key=receiver.pub)
cipher_signed_with_noise

'[\n"<rsachunkrsa>AyXdWqrk+0k4iuX2+t8eA&w)v3m,u0rp[@+x)?Ab@LJ?ptw\\nIFs]uzYrT8\\\\Ss3o:v_QuE(6,s%e>&[7S6JC.>~Jv_lHnfa{XHl*8\\"!J w{b\\n0qp$^a2N0F\'D#%1Nx\\n$gQ:o\\n}kqW\\n[jsE]c&\\"(8G?lGS</rsachunkrsa>"\n]'

##### Shortcut

In [67]:
cipher_signed == sender.sends(plain_message, _to=receiver.pub)

True

In [68]:
cipher_signed == sender.sends(plain_message, _to=receiver)

True

In [69]:
cipher_signed_with_noise_bis = sender.sends(plain_message, _to=receiver, _with_noises=True)
cipher_signed_with_noise_bis


'NoIsYnOiSy[\n"<rsachunkrsa>D-i^ko\\"\\"9&`f=9hr-NW{Fq/)g|Aaf9Lq-f]J@{5}|P0WEguAE,:r,]eOHR@sD-HnFZU~AgH AUXk\\"Cv0XB ~f9a2AWs;FRcc`-D\\\\:y{$iH!@u@NsUlMug*NQ|Ee\\\\jkjF6T@i3j0sUvNU%9YQG/dx<H{2</rsachunkrsa>"\n]NoIsYnOiSy'

In [70]:
cipher_signed_with_noise_bis[10:-10] # trim NoIsYnOiSy

'[\n"<rsachunkrsa>D-i^ko\\"\\"9&`f=9hr-NW{Fq/)g|Aaf9Lq-f]J@{5}|P0WEguAE,:r,]eOHR@sD-HnFZU~AgH AUXk\\"Cv0XB ~f9a2AWs;FRcc`-D\\\\:y{$iH!@u@NsUlMug*NQ|Ee\\\\jkjF6T@i3j0sUvNU%9YQG/dx<H{2</rsachunkrsa>"\n]'

In [71]:
cipher_signed_with_noise == cipher_signed_with_noise_bis[10:-10]

False

##### Q: Do you know why is it `False`?

#### Decryption

In [72]:
signed_message = receiver.decrypt(cipher_signed)
out = receiver.encrypt(signed_message, key=sender.pub) # remove signature

if isinstance(sender.pub, crypto.RSAKey):
    out = crypto.read_from_rsa(out)
out

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

In [73]:
signed_message_with_noise = receiver.decrypt(cipher_signed_with_noise)
message_with_noise = receiver.encrypt(signed_message_with_noise, key=sender.pub) # unsign
if isinstance(sender.pub, crypto.RSAKey):
    message_with_noise = crypto.read_from_rsa(message_with_noise)
utils.remove_noises(message_with_noise)

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

In [74]:
trimmed_cipher_signed_with_noise = cipher_signed_with_noise_bis[10:-10]

signed_message_with_noise = receiver.decrypt(trimmed_cipher_signed_with_noise)
message_with_noise = receiver.encrypt(signed_message_with_noise, key=sender.pub) # unsign

if isinstance(sender.pub, crypto.RSAKey):
    message_with_noise = crypto.read_from_rsa(message_with_noise)
    
utils.remove_noises(message_with_noise)

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

##### Shortcut

In [75]:
receiver.gets(cipher_signed, _from=sender.pub)

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

In [76]:
utils.remove_noises(receiver.gets(cipher_signed_with_noise, _from=sender))

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

In [77]:
receiver.gets(cipher_signed_with_noise_bis, _from=sender)

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

#### Attacks

##### Reading Attack


In [78]:
out = hacker.decrypt(cipher_signed)
if isinstance(hacker.pub, crypto.RSAKey):
    out = crypto.read_from_rsa(out)
out

')R@Ik:;sEWj.\'E#fv\\8jtYW|&e6lJ"4kMY&2S"le5COj\nEB!t!=W5;]y-Z`PuzYt3(nH)6Ljt)d1'

In [79]:
out = hacker.encrypt(cipher_signed, key=sender.pub)

if isinstance(hacker.pub, crypto.RSAKey):
    out = crypto.read_from_rsa(out)
out

"Fy|R5/pTP&v{ktMTfht8 Q&GG_|2^wU0o\\!*{U8DZYb,7,/8C 2Z$k1';3b=Wg-:kBC5EN,wE~#W"

In [80]:
out = hacker.encrypt(cipher_signed, key=receiver.pub)
if isinstance(receiver.pub, crypto.RSAKey):
    out = crypto.read_from_rsa(out)
out

"0y[P#&f(A?J5iNF\\d/Mx1VzZ*v^.vA<,gY-E{mqcX1~KYA#6y{brJ2Fq$-\t2yPqPq]0'+;~=~NPf"

In [81]:
out = hacker.sends(cipher_signed, _to=receiver.pub)
if isinstance(receiver.pub, crypto.RSAKey):
    out = crypto.read_from_rsa(out)
out

'$nni2V!kcm9 x} "\'%lgM:M"S^m<^<C~`o:08ad\n+{Q\n?S0`Pfcg6[nattT|[s6M9xic1d&;!s/3'

In [82]:
hacker.gets(cipher_signed, _from=sender)

'81/nJ(+44o1O,!>8ge4$3($J&P}L%)x|R.]Yd{,V;OS-?LJ~>\\<wa6c!uZ-iNzqeN`DN4YNr,]>t'

In [83]:
hacker.gets(cipher_signed, _from=bob)

'$nni2V!kcm9 x} "\'%lgM:M"S^m<^<C~`o:08ad\n+{Q\n?S0`Pfcg6[nattT|[s6M9xic1d&;!s/3'

##### Sending Attack


In [84]:
print(f"""
Original:
========
{plain_message}

Scam:
=====
{scam_message}
"""
)


Original:
Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)

Scam:
=====
Hello Bob! This is Alice. Please send me $1000 to this account: (0987654321)



In [85]:
non_signed_scam = hacker.encrypt(scam_message, key=receiver.pub)

receiver.gets(non_signed_scam, _from=sender)

'A2^zvkWvZiAbm5HN]EvSgPZj:)H5PJ*E\\/~:_C-gF\n.u*dwFNz\\kBSqSS05+z/Xs`cMdDVb1hv=!'

In [86]:
non_signed_scam = hacker.encrypt(scam_message, key=sender.pub)

receiver.gets(non_signed_scam, _from=sender)

'2`l0P/F`S"[V#|%X3mFXF}UJIpcEDX\\Y`hRyhbqD\\5eF5mhE=5`!j:scwN0>HwBv}(`&uvg=sEfW'

In [87]:
signed_scam = hacker.sends(scam_message, _to=receiver)

receiver.gets(signed_scam, _from=sender)

'7} t%<|l}<F_"q#m+ZtHQsX.Rv{e]U\\T|iF5(:s\'6X6{|*N{90kqr3vV\'i!C#y9}a>gFLP>AUWnt'

##### Possible Weakness

In [88]:
_, leaked_sender_priv = sender._Node__keys  # key of alice = expected sender

fake_signed_scam = hacker.encrypt(scam_message, leaked_sender_priv)
cipher_fake_signed_scam = hacker.encrypt(fake_signed_scam, receiver.pub)
out = cipher_fake_signed_scam

if isinstance(receiver.pub, crypto.RSAKey):
    out = crypto.read_from_rsa(out)
out

'"Psc\t23&Fm("OQU7NGZLu\nIz7;i?\\F;<bmR\n+%@l!0iR7"&qZ&?V`+f~8+v\n]FAPNq#mb@/F#vNW'

In [89]:
receiver.gets(cipher_fake_signed_scam, _from=sender)

'Hello Bob! This is Alice. Please send me $1000 to this account: (0987654321)'

# Blockchain


In [90]:
from bcbcpy import blockchain

## Recap: Data Structure

### List - Array

### Linked List

## Block

In [91]:
from bcbcpy.blockchain.block import InitialBlock, Block

### Initial Block

In [92]:
init_block = InitialBlock(initial_data="initial data")

In [93]:
init_block.data

'initial data'

In [94]:
init_block.hash

'00002d64bd7129caa2aa8ec458081519ac26034158e60962f6fd1428e22b1036'

In [95]:
init_block

{
    "prev_hash": "",
    "hash": "00002d64bd7129caa2aa8ec458081519ac26034158e60962f6fd1428e22b1036",
    "data": "initial data",
    "nonce": 26206
}

### Block Data

In [96]:
second_block = Block(data="second data", prev_block= init_block)
second_block

{
    "prev_hash": "00002d64bd7129caa2aa8ec458081519ac26034158e60962f6fd1428e22b1036",
    "hash": "ac19b9c98cc9a1655bd66662c552b8667c2140a451ce635b24aa2fdf34555aea",
    "data": "second data",
    "nonce": 0
}

In [97]:
second_block.is_valid()

False

In [98]:
second_block.mine()
second_block

{
    "prev_hash": "00002d64bd7129caa2aa8ec458081519ac26034158e60962f6fd1428e22b1036",
    "hash": "0000f1a7cc9b3335858fdc615b274e07568931de65e82fb991ac257e168b89c1",
    "data": "second data",
    "nonce": 12005
}

In [99]:
second_block.is_valid()

True

In [100]:
third_block = Block("third data", prev_block= second_block)
third_block

{
    "prev_hash": "0000f1a7cc9b3335858fdc615b274e07568931de65e82fb991ac257e168b89c1",
    "hash": "f94643bbf3edffb31b549ce63a5adbd4542b7e3b6d45f32ae69da4f9bc87b6f7",
    "data": "third data",
    "nonce": 0
}

In [101]:
third_block.mine()
third_block

{
    "prev_hash": "0000f1a7cc9b3335858fdc615b274e07568931de65e82fb991ac257e168b89c1",
    "hash": "0000eecdfce497ff979154b54ddb7f8fe0114683c0102afb900cd93497951e80",
    "data": "third data",
    "nonce": 2917
}

In [102]:
forth_block = Block("forth data", third_block)
forth_block.mine()
forth_block

{
    "prev_hash": "0000eecdfce497ff979154b54ddb7f8fe0114683c0102afb900cd93497951e80",
    "hash": "00003e75b774fcf431090692c595e06fa5d05bc7730a63673e4e4304eb4faf6a",
    "data": "forth data",
    "nonce": 101056
}

In [103]:
fifth_block = Block("fifth block", forth_block)
fifth_block.mine()
fifth_block

{
    "prev_hash": "00003e75b774fcf431090692c595e06fa5d05bc7730a63673e4e4304eb4faf6a",
    "hash": "0000d8cb89717a6b1904bf4f44aab11529b3ee378d0256620f608c3dd73694b9",
    "data": "fifth block",
    "nonce": 34705
}

In [104]:
init_block.is_valid(), second_block.is_valid(), third_block.is_valid(), forth_block.is_valid(), fifth_block.is_valid()

(True, True, True, True, True)

### Making some change in past data

In [105]:
third_block.data = "not third data"
third_block

{
    "prev_hash": "0000f1a7cc9b3335858fdc615b274e07568931de65e82fb991ac257e168b89c1",
    "hash": "66ad1a8ee995ddbcaaf131585222c10a396a960f0c441578d5613417fa5d107a",
    "data": "not third data",
    "nonce": 2917
}

In [106]:
init_block.is_valid(), second_block.is_valid(), third_block.is_valid(), forth_block.is_valid(), fifth_block.is_valid()

(True, True, False, False, False)

In [107]:
third_block.data = "third data"
init_block.is_valid(), second_block.is_valid(), third_block.is_valid(), forth_block.is_valid(), fifth_block.is_valid()

(True, True, True, True, True)

In [108]:
third_block.data = "not third data"
third_block.mine()
third_block.is_valid()

True

In [109]:
init_block.is_valid(), second_block.is_valid(), third_block.is_valid(), forth_block.is_valid(), fifth_block.is_valid()

(True, True, True, False, False)

In [110]:
third_block.data = "third data"
third_block.mine()
init_block.is_valid(), second_block.is_valid(), third_block.is_valid(), forth_block.is_valid(), fifth_block.is_valid()

(True, True, True, True, True)

## Blockchain

A submodule is available at 
```python
from bcbcpy.blockchain import chain
```

### Root

In [111]:
blockchain.RootChain()

{
    "root_block": {
        "hash": "0000a456e7b5a5eb059e721fb431436883143101275c4077f83fe70298f5623d",
        "data": ""
    }
}

In [112]:
initial_data = {"INITIAL AGREEMENT": {"Bob": 1000, "Alice": 500}}

initial_data_string = utils.obj2txt(initial_data, indent=4)
root_chain = blockchain.RootChain(initial_data_string, difficulty=4)

root_chain

{
    "root_block": {
        "hash": "0000579da420f2b62497716cc60e95a0f7ad46336c42062b7485a27fd91d99f0",
        "data": "{\n    \"INITIAL ..."
    }
}

### Chain

In [113]:
prev_block = root_chain.last_block
new_block = Block("new block", prev_block)
new_block

{
    "prev_hash": "0000579da420f2b62497716cc60e95a0f7ad46336c42062b7485a27fd91d99f0",
    "hash": "73095d20a662a26d26eec210c0dc51c34816098cdab05f941487ee5e8133d6ad",
    "data": "new block",
    "nonce": 0
}

---

```python
root_chain.add_block(
  new_block
)
```

---
```sh
 ---------------------------------------------------------------------------
 AssertionError                            Traceback (most recent call last)
 /var/folders/x4/45r2t3bx5l3dc_b3tm_yz7380000gn/T/ipykernel_26354/943714534.py in <cell line: 1>()
 ----> 1 root_chain.add_block(
       2     new_block
       3 )

 ~/Desktop/BootCamp-BlockChain-and-Python/src/bcbcpy/blockchain/chain.py in add_block(self, block)
      26             self.last_block is block.prev_block
      27         ), "Incompatible block to the head block of the chain."
 ---> 28         assert block.is_valid(), "Block is not valid."
      29         self.last_block = block
      30 

 AssertionError: Block is not valid.
 
```
---

In [114]:
new_block.mine() # validation
new_block

{
    "prev_hash": "0000579da420f2b62497716cc60e95a0f7ad46336c42062b7485a27fd91d99f0",
    "hash": "00007b581d241fea3fb8bf4c7c5c1941af9abb3a5ce05abb97b4521d34bab727",
    "data": "new block",
    "nonce": 2823
}

In [115]:
root_chain.add_block(
    new_block
)

In [116]:
root_chain

{
    "root_block": {
        "hash": "0000579da420f2b62497716cc60e95a0f7ad46336c42062b7485a27fd91d99f0",
        "data": "{\n    \"INITIAL ..."
    },
    "last_block": {
        "hash": "00007b581d241fea3fb8bf4c7c5c1941af9abb3a5ce05abb97b4521d34bab727",
        "data": "new block"
    }
}

In [117]:
chain = blockchain.Chain(root_chain=root_chain, info="local network")
chain

{
    "root_block": {
        "hash": "000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9",
        "data": "local network"
    }
}

## Transaction


In [118]:
from bcbcpy.blockchain.block import transaction

### Nodes setup

In [119]:
from bcbcpy.blockchain.node import Node

alice = Node(crypto.RSAPairKeys.generate_pairs(), "Alice")
bob = Node(crypto.RSAPairKeys.generate_pairs(), "Bob")
other_person_in_the_network = Node(crypto.RSAPairKeys.generate_pairs())

### Transaction Data

In [120]:
assets = 1000
sender = bob
receiver = alice

In [121]:
receiver_id = receiver.id
receiver_pub = receiver.pub

prev_block = chain.last_block

In [122]:
transaction_data = transaction.TransactionData(
    assets, sender, receiver_id, receiver_pub, prev_block
)

print(transaction_data)

{'timestamp': '2023-05-04T03:18:29.299250', 'sender_id': 'Bob', 'receiver_id': 'Alice', 'assets': 1000, 'prev_hash': '000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9', 'sender_confirmation': '[\n"<rsachunkrsa>  EcD0HM&E@Kb+y fE\\t>#s`F?)R}UU_~*[6Bch,)~{vK2yf1i?pBAF2dn7I#{D7d56h.oA/p&tTg7urbPvx&\\n^ca+\\t>pwDOZ05oU6iT&jz_I.}yte\\t@YE/38&maGrf*xF>~`i</rsachunkrsa>"\n]'}


### Transaction Block

In [123]:
transaction_block = blockchain.TransactionBlock(transaction_data)
transaction_block

{
    "prev_hash": "000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9",
    "hash": "65222279c494ce87948c3fd34ee3b37436b482a9a0145252baf502d208aec57e",
    "data": "{'timestamp': '2023-05-04T03:18:29.299250', 'sender_id': 'Bob', 'receiver_id': 'Alice', 'assets': 1000, 'prev_hash': '000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9', 'sender_confirmation': '[\\n\"<rsachunkrsa>  EcD0HM&E@Kb+y fE\\\\t>#s`F?)R}UU_~*[6Bch,)~{vK2yf1i?pBAF2dn7I#{D7d56h.oA/p&tTg7urbPvx&\\\\n^ca+\\\\t>pwDOZ05oU6iT&jz_I.}yte\\\\t@YE/38&maGrf*xF>~`i</rsachunkrsa>\"\\n]'}",
    "nonce": 0
}

### Verification


#### Anyone in the network

In [124]:
data = transaction_block.data

print(data)


{'timestamp': '2023-05-04T03:18:29.299250', 'sender_id': 'Bob', 'receiver_id': 'Alice', 'assets': 1000, 'prev_hash': '000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9', 'sender_confirmation': '[\n"<rsachunkrsa>  EcD0HM&E@Kb+y fE\\t>#s`F?)R}UU_~*[6Bch,)~{vK2yf1i?pBAF2dn7I#{D7d56h.oA/p&tTg7urbPvx&\\n^ca+\\t>pwDOZ05oU6iT&jz_I.}yte\\t@YE/38&maGrf*xF>~`i</rsachunkrsa>"\n]'}


In [125]:
raise Warning("Sender confirmation below needs to be set manually due to dirty rsa output")

Warning: Sender confirmation below needs to be set manually due to dirty rsa output

In [126]:
sender_confirmation="""[\n"<rsachunkrsa>  EcD0HM&E@Kb+y fE\\t>#s`F?)R}UU_~*[6Bch,)~{vK2yf1i?pBAF2dn7I#{D7d56h.oA/p&tTg7urbPvx&\\n^ca+\\t>pwDOZ05oU6iT&jz_I.}yte\\t@YE/38&maGrf*xF>~`i</rsachunkrsa>"\n]""" 
print(sender_confirmation)

[
"<rsachunkrsa>  EcD0HM&E@Kb+y fE\t>#s`F?)R}UU_~*[6Bch,)~{vK2yf1i?pBAF2dn7I#{D7d56h.oA/p&tTg7urbPvx&\n^ca+\t>pwDOZ05oU6iT&jz_I.}yte\t@YE/38&maGrf*xF>~`i</rsachunkrsa>"
]


In [127]:
confirmation_string = other_person_in_the_network.encrypt(sender_confirmation, key= sender.pub)
if isinstance(sender.pub, crypto.RSAKey):
    confirmation_string = crypto.read_from_rsa(confirmation_string)
print(confirmation_string)

[['assets',	1000],
['receiver_pub',	(7313, 4457)],
['prev_hash',	000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9]]


In [128]:
print(receiver.pub)
print(chain.hash)

(7313, 4457)
000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9


### Block Validation

In [129]:
transaction_block.is_valid()

False

In [130]:
transaction_block.mine()
transaction_block

{
    "prev_hash": "000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9",
    "hash": "000055561c459c463beba5f400067485cd6fd68ca3f860f7f35a49be7c095e4f",
    "data": "{'timestamp': '2023-05-04T03:18:29.299250', 'sender_id': 'Bob', 'receiver_id': 'Alice', 'assets': 1000, 'prev_hash': '000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9', 'sender_confirmation': '[\\n\"<rsachunkrsa>  EcD0HM&E@Kb+y fE\\\\t>#s`F?)R}UU_~*[6Bch,)~{vK2yf1i?pBAF2dn7I#{D7d56h.oA/p&tTg7urbPvx&\\\\n^ca+\\\\t>pwDOZ05oU6iT&jz_I.}yte\\\\t@YE/38&maGrf*xF>~`i</rsachunkrsa>\"\\n]'}",
    "nonce": 57254
}

In [131]:
transaction_block.is_valid()

True

### Adding to Chain

In [132]:
chain.add_block(transaction_block)
chain

{
    "root_block": {
        "hash": "000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9",
        "data": "local network"
    },
    "last_block": {
        "hash": "000055561c459c463beba5f400067485cd6fd68ca3f860f7f35a49be7c095e4f",
        "data": "{'timestamp': '..."
    }
}

## Second Transaction


In [133]:
assets = 500
sender = alice
receiver = bob
prev_block = chain.last_block

In [134]:
transaction_block = sender.make_transaction_block(
    assets=assets,
    to=receiver,
    prev_block=prev_block,
)

### Verification ...

In [135]:
data = transaction_block.data
print(data)

{'timestamp': '2023-05-04T03:19:09.156649', 'sender_id': 'Alice', 'receiver_id': 'Bob', 'assets': 500, 'prev_hash': '000055561c459c463beba5f400067485cd6fd68ca3f860f7f35a49be7c095e4f', 'sender_confirmation': '[\n"<rsachunkrsa> 5S*vJW]</}~!C#|rb9Gdp203+N75_7#(6D~rNm-aHE1c=hNo3ozk,_A4UZ)_eY\\tem/\\"}:wx%oLVwAPknU1lt*DF-(68+1U^xXvE/*HlqG#ti:]f?2l\\n^TG77\']Kmp+,u/z</rsachunkrsa>"\n]'}


In [136]:
raise Warning("Sender confirmation below needs to be set manually due to dirty rsa output")

Warning: Sender confirmation below needs to be set manually due to dirty rsa output

In [137]:
sender_confirmation="""[\n"<rsachunkrsa> 5S*vJW]</}~!C#|rb9Gdp203+N75_7#(6D~rNm-aHE1c=hNo3ozk,_A4UZ)_eY\\tem/\\"}:wx%oLVwAPknU1lt*DF-(68+1U^xXvE/*HlqG#ti:]f?2l\\n^TG77\']Kmp+,u/z</rsachunkrsa>"\n]""" 
confirmation_string = other_person_in_the_network.encrypt(sender_confirmation, key= sender.pub)
if isinstance(sender.pub, crypto.RSAKey):
    confirmation_string = crypto.read_from_rsa(confirmation_string)
confirmation = utils.txt2obj(confirmation_string)
print(confirmation)

[['assets',	500],
['receiver_pub',	(7313, 4603)],
['prev_hash',	000055561c459c463beba5f400067485cd6fd68ca3f860f7f35a49be7c095e4f]]


In [138]:
print(receiver.pub)
print(chain.hash)

(7313, 4603)
000055561c459c463beba5f400067485cd6fd68ca3f860f7f35a49be7c095e4f


### Adding to Chain

In [139]:
transaction_block.mine() # validation
chain.add_block(transaction_block)

In [140]:
chain

{
    "root_block": {
        "hash": "000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9",
        "data": "local network"
    },
    "...": "...",
    "last_block": {
        "hash": "0000a07716c72426eccb891b3be357269294239927c637d5ace03bcab696d617",
        "data": "{'timestamp': '..."
    }
}